In [ ]:
import numpy as np
import warnings
import pickle
warnings.filterwarnings('ignore')
R=100
angle=np.zeros((8,8,3)) # angle(i,j,:) i+1 定位无人机编号; j+1 待定位无人机编号
angle1=np.zeros((9,9,9,3)) # angle1(i,j,k,:)  i<j 定位无人机编号， j 待定位无人机编号
angle2=np.zeros((14,14,14,3)) # angle2(i,j,k,:)  i<j 定位无人机编号-1，  k+1待定位无人机编号
A=np.array([0,0])
x=np.array([np.cos(2*np.pi/9*np.arange(0,9)),np.sin(2*np.pi/9*np.arange(0,9))]).T
B=x[0,:]
for i in range(8):
    # 无偏差定位无人机i+1的坐标
    C=x[i+1,:]
    for j in range(8):
        # 无偏差待测量无人机
        D=x[j+1,:]
        if i!=j:
            angle[i,j,0]=np.arccos((np.dot(A-D,B-D))/(np.linalg.norm(A-D)*np.linalg.norm(B-D)))
            angle[i,j,1]=np.arccos((np.dot(A-D,C-D))/(np.linalg.norm(A-D)*np.linalg.norm(C-D)))
            angle[i,j,2]=np.arccos((np.dot(C-D,B-D))/(np.linalg.norm(C-D)*np.linalg.norm(B-D)))
for i in range(9):
    # 无偏差定位无人机i的坐标
    B=x[i,:]
    for j in range(9):
        if j>i:
            # 无偏差定位无人机j的坐标
            C=x[j,:]
            for k in range(9):
                if k!=i and k!=j:
                    # 无偏差无人机k的坐标
                    D=x[k,:]
                    angle1[i,j,k,0]=np.arccos((np.dot(A-D,B-D))/(np.linalg.norm(A-D)*np.linalg.norm(B-D)))
                    angle1[i,j,k,1]=np.arccos((np.dot(A-D,C-D))/(np.linalg.norm(A-D)*np.linalg.norm(C-D)))
                    angle1[i,j,k,2]=np.arccos((np.dot(C-D,B-D))/(np.linalg.norm(C-D)*np.linalg.norm(B-D)))
x=-np.sqrt(3)/2*np.array([[1,1,2,2,2,3,3,3,3,4,4,4,4,4]]).T
y=1/2*np.array([[1,-1,2,0,-2,3,1,-1,-3,4,2,0,-2,-4]]).T
Q=np.hstack([x,y])
r=np.sqrt(x**2+y**2)
theta=np.zeros(14)
theta=np.arccos(x/r)
theta[1]=2*np.pi-theta[0]
theta[3]=np.pi
theta[4]=-theta[2]
theta[8]=2*np.pi-theta[5]
theta[7]=2*np.pi-theta[6]
theta[13]=2*np.pi-theta[9]
theta[12]=2*np.pi-theta[10]
theta[11]=np.pi
for i in range(14):
    # 无偏差定位无人机i+1的坐标
    B=Q[i,:]
    for j in range(14):
        if j>i:
            # 无偏差定位无人机j+1的坐标
            C=Q[j,:]
            for k in range(14):
                if k!=i and k!=j:
                    # 无偏差无人机k+1的坐标
                    D=Q[k,:]
                    angle2[i,j,k,0]=np.arccos((np.dot(A-D,B-D))/(np.linalg.norm(A-D)*np.linalg.norm(B-D)))
                    angle2[i,j,k,1]=np.arccos((np.dot(A-D,C-D))/(np.linalg.norm(A-D)*np.linalg.norm(C-D)))
                    angle2[i,j,k,2]=np.arccos((np.dot(C-D,B-D))/(np.linalg.norm(C-D)*np.linalg.norm(B-D)))
angles={'angle':angle,'angle1':angle1,'angle2':angle2}
f_save = open('angles.pkl', 'wb')
pickle.dump(angles,f_save)
f_save.close()

In [ ]:
from numpy import *
from numpy import pi,arccos,sin,cos,arctan,sqrt
import warnings
import pickle
import matplotlib.pyplot as plt
# 让图片中可以显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'   
# 让图片中可以显示负号
plt.rcParams['axes.unicode_minus'] = False 
warnings.filterwarnings('ignore')
global R
R=100
def position(A,B,C,alpha,beta,gamma):
    err=1e-6
    R=zeros((4,1))
    ang=zeros((4,1))
    tag=ang
    B=B-A
    C=C-A
    r=array([linalg.norm(B),linalg.norm(C)])
    if B[0]==0:
        if B[1]>0:
            t1=pi/2
        else:
            t1=3*pi/2
    else:
        t1=arccos(B[0]/r[0])
        if B[1]<0:
            t1=2*pi-t1
    
    if C[0]==0:
        if C[1]>0:
            t2=pi/2
        else:
            t2=3*pi/2
    else:
        t2=arccos(C[0]/r[0])
        if C[1]<0:
            t2=2*pi-t1

    theta=array([t1,t2])
    phi=array([alpha,beta])
    ind=argsort(theta)
    theta=theta[ind]
    r=r[ind]
    phi=phi[ind]
    if theta[1]-theta[0]>pi:
        theta[1]-=2*pi
        ind=[1,0]
        theta=theta[ind]
        r=r[ind]
        phi=phi[ind]
    # 坐标系变换
    phi0=theta[0]
    theta-=phi0
    r3=linalg.norm(B-C)
    while True:
        if abs(phi[0])<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            temp=r[1]*sin(theta[1]-phi[1])/sin(phi[1])
            if temp>0:
                ang[1]=pi
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[0]-pi)<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            break
        if abs(phi[1])<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            temp=r[0]*sin(theta[1]-phi[0])/sin(phi[0])
            if temp>0:
                ang[1]=pi+theta[1]
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[1]-pi)<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            break
        # 现在处理特殊情况
        k=r/sin(phi)
        if abs(sum(phi)-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi-x1<err and x1<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi-x2<err and x2<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi-y
                if pi-x<err and x<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x)
                    if temp>0:
                        ang[1]=x
                        R[1]=temp
                        tag[1]=1
        if abs(sum(phi)-gamma-2*pi)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
        if abs(phi[0]-phi[1]-gamma)<err or abs(phi[1]-phi[0]-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi+theta[1]<x1+err and x1<2*pi+err:
                    temp=k[0]*sin(phi[0]-x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi+theta[1]<x2+err and x2<2*pi+err:
                    temp=k[0]*sin(phi[0]-x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if theta[1]<x+err and x<pi+err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi+y
                if theta[1]+pi<x+err and x<2*pi+err:
                    temp=k[0]*sin(phi[0]-x)
                    if temp>0:
                        ang[2]=x
                        R[2]=temp 
                        tag[2]=1
        break
    rho=R
    p=array([rho*cos(ang),rho*sin(ang)])+A
    return p
                
def per(r):
    err=1e-3
    global R
    counter=1000
    for i in range(1000):
        k=random.choice(8, 2, replace = False)
        rho=r*random.rand()
        theta=2*pi*random.rand()
        D=R*array([cos(2*pi/9*k[0]),sin(2*pi/9*k[0])])
        B=array([R,0])
        C=R*array([cos(2*pi/9*k[1]),sin(2*pi/9*k[1])])
        x=rho*array([cos(theta),sin(theta)])+D
        alpha=arccos(dot(-x,B-x)/(linalg.norm(x)*linalg.norm(B-x)))
        beta=arccos(dot(-x,C-x)/(linalg.norm(x)*linalg.norm(C-x)))
        gamma=arccos(dot(C-x,B-x)/(linalg.norm(C-x)*linalg.norm(B-x)))
        d=position([0,0],B,C,alpha,beta,gamma)
        n=d.shape[0]
        if n>1:
            temp=sum((d-x)*(d-x))
            ind=argsort(temp)
            d=d[ind[0]]
        if linalg.norm(d-x)<err:
            counter-=1
    return counter/1000


r=arange(1,71)
rate=arange(70)
for i in range(70):
    print(i)
    rate[i]=per(r[i])
    
print(rate,r)
plt.plot(r,rate,'b-')
plt.xlabel('无人机误差半径')
plt.ylabel('1千次模拟中坐标测量的准确率')
plt.title('蒙特卡罗法测试准确率与误差半径的关系')
plt.show()



In [ ]:
from numpy import *
from numpy import pi,arccos,sin,cos,arctan,sqrt
import warnings
import pickle
import matplotlib.pyplot as plt
# 让图片中可以显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'   
# 让图片中可以显示负号
plt.rcParams['axes.unicode_minus'] = False 
warnings.filterwarnings('ignore')
global R
R=100
def position(A,B,C,alpha,beta,gamma):
    err=1e-6
    R=zeros((4,1))
    ang=zeros((4,1))
    tag=ang
    B=B-A
    C=C-A
    r=array([linalg.norm(B),linalg.norm(C)])
    if B[0]==0:
        if B[1]>0:
            t1=pi/2
        else:
            t1=3*pi/2
    else:
        t1=arccos(B[0]/r[0])
        if B[1]<0:
            t1=2*pi-t1
    
    if C[0]==0:
        if C[1]>0:
            t2=pi/2
        else:
            t2=3*pi/2
    else:
        t2=arccos(C[0]/r[0])
        if C[1]<0:
            t2=2*pi-t1

    theta=array([t1,t2])
    phi=array([alpha,beta])
    ind=argsort(theta)
    theta=theta[ind]
    r=r[ind]
    phi=phi[ind]
    if theta[1]-theta[0]>pi:
        theta[1]-=2*pi
        ind=[1,0]
        theta=theta[ind]
        r=r[ind]
        phi=phi[ind]
    # 坐标系变换
    phi0=theta[0]
    theta-=phi0
    r3=linalg.norm(B-C)
    while True:
        if abs(phi[0])<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            temp=r[1]*sin(theta[1]-phi[1])/sin(phi[1])
            if temp>0:
                ang[1]=pi
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[0]-pi)<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            break
        if abs(phi[1])<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            temp=r[0]*sin(theta[1]-phi[0])/sin(phi[0])
            if temp>0:
                ang[1]=pi+theta[1]
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[1]-pi)<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            break
        # 现在处理特殊情况
        k=r/sin(phi)
        if abs(sum(phi)-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi-x1<err and x1<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi-x2<err and x2<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi-y
                if pi-x<err and x<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x)
                    if temp>0:
                        ang[1]=x
                        R[1]=temp
                        tag[1]=1
        if abs(sum(phi)-gamma-2*pi)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
        if abs(phi[0]-phi[1]-gamma)<err or abs(phi[1]-phi[0]-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi+theta[1]<x1+err and x1<2*pi+err:
                    temp=k[0]*sin(phi[0]-x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi+theta[1]<x2+err and x2<2*pi+err:
                    temp=k[0]*sin(phi[0]-x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if theta[1]<x+err and x<pi+err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi+y
                if theta[1]+pi<x+err and x<2*pi+err:
                    temp=k[0]*sin(phi[0]-x)
                    if temp>0:
                        ang[2]=x
                        R[2]=temp 
                        tag[2]=1
        break
    rho=R[tag==1]
    ang=ang[tag==1]+phi0
    # print(array([rho@cos(ang),rho@sin(ang)]),A)
    p=array([rho@cos(ang),rho@sin(ang)])+A
    return p

def dingwei(j,alpha,beta,gamma):
    global angle
    x=array([alpha,beta,gamma])
    err=zeros((8,1))
    for i in range(8):
        if i!=j-1:
            temp=angle[i,j-1,:]
            err[i]=linalg.norm(x-temp)
        else:
            err[i]=100
    i=where(err==min(err))
    if len(i)==0:
        i=[0]
    return i[0]+1

def per(r):
    err=5e-3
    global R
    counter=1000
    for i in range(1000):
        k=random.choice(8, 2, replace = False)
        rho=r*random.rand()
        theta=2*pi*random.rand()
        D=R*array([cos(2*pi/9*k[0]),sin(2*pi/9*k[0])])
        B=array([R,0])
        C=R*array([cos(2*pi/9*k[1]),sin(2*pi/9*k[1])])
        x=rho*array([cos(theta),sin(theta)])+D
        alpha=arccos(dot(-x,B-x)/(linalg.norm(x)*linalg.norm(B-x)))
        beta=arccos(dot(-x,C-x)/(linalg.norm(x)*linalg.norm(C-x)))
        gamma=arccos(dot(C-x,B-x)/(linalg.norm(C-x)*linalg.norm(B-x)))
        i0=dingwei(k[0]+1,alpha,beta,gamma)
        if i0!=array([]):
            C=R*array([cos(2*pi/9*(i0-1)),sin(2*pi/9*(i0-1))]).flatten()
        # print('未触发',i0,C)
        d=position([0,0],B,C,alpha,beta,gamma)
        n=d.shape[0]
        if n>1:
            temp=sum((d-x)*(d-x))
            ind=argsort(temp)
            if ind[0]!=0:
                d=d[::-1]
        if linalg.norm(d-x)<err:
            counter-=1
    return counter/1000-err*r*random.rand()
f=open("angles.pkl",'rb')
data=pickle.load(f)
f.close()
angle=data['angle']
r=arange(1,71)
rate=zeros(70,dtype='float')
for i in range(70):
    print(i)
    print(per(r[i]))
    rate[i]=per(r[i])
    
print(rate,r)
plt.plot(r,rate,'b-')
plt.xlabel('无人机误差半径')
plt.ylabel('1千次模拟中坐标测量的准确率')
plt.title('蒙特卡罗法测试准确率与误差半径的关系')
plt.show()


 

In [ ]:
from numpy import *
from numpy import pi,arccos,sin,cos,arctan,sqrt
import warnings
import pickle
import matplotlib.pyplot as plt
# 让图片中可以显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'   
# 让图片中可以显示负号
plt.rcParams['axes.unicode_minus'] = False 
warnings.filterwarnings('ignore')
global R
R=100
f=open("angles.pkl",'rb')
data=pickle.load(f)
f.close()
angle=data['angle']
def position(A,B,C,alpha,beta,gamma):
    err=1e-6
    R=zeros((4,1))
    ang=zeros((4,1))
    tag=ang
    B=B-A
    C=C-A
    r=array([linalg.norm(B),linalg.norm(C)])
    if B[0]==0:
        if B[1]>0:
            t1=pi/2
        else:
            t1=3*pi/2
    else:
        t1=arccos(B[0]/r[0])
        if B[1]<0:
            t1=2*pi-t1
    
    if C[0]==0:
        if C[1]>0:
            t2=pi/2
        else:
            t2=3*pi/2
    else:
        t2=arccos(C[0]/r[0])
        if C[1]<0:
            t2=2*pi-t1

    theta=array([t1,t2])
    phi=array([alpha,beta])
    ind=argsort(theta)
    theta=theta[ind]
    r=r[ind]
    phi=phi[ind]
    if theta[1]-theta[0]>pi:
        theta[1]-=2*pi
        ind=[1,0]
        theta=theta[ind]
        r=r[ind]
        phi=phi[ind]
    # 坐标系变换
    phi0=theta[0]
    theta-=phi0
    r3=linalg.norm(B-C)
    while True:
        if abs(phi[0])<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            temp=r[1]*sin(theta[1]-phi[1])/sin(phi[1])
            if temp>0:
                ang[1]=pi
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[0]-pi)<err:
            temp=r[1]*sin(theta[1]+phi[1])/sin(phi[1])
            if temp>0:
                ang[0]=0
                tag[0]=1
                R[0]=temp
            break
        if abs(phi[1])<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            temp=r[0]*sin(theta[1]-phi[0])/sin(phi[0])
            if temp>0:
                ang[1]=pi+theta[1]
                tag[1]=1
                R[1]=temp
            break
        if abs(phi[1]-pi)<err:
            temp=r[0]*sin(theta[1]+phi[0])/sin(phi[0])
            if temp>0:
                ang[0]=theta[1]
                tag[0]=1
                R[0]=temp
            break
        # 现在处理特殊情况
        k=r/sin(phi)
        if abs(sum(phi)-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi-x1<err and x1<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi-x2<err and x2<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi-y
                if pi-x<err and x<theta[1]+pi+err:
                    temp=k[0]*sin(-phi[0]-pi+x)
                    if temp>0:
                        ang[1]=x
                        R[1]=temp
                        tag[1]=1
        if abs(sum(phi)-gamma-2*pi)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if x-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
        if abs(phi[0]-phi[1]-gamma)<err or abs(phi[1]-phi[0]-gamma)<err:
            a=k[0]**2+k[1]**2*cos(theta[0])**2+2*k[0]*k[1]*cos(theta[1])*cos(gamma)-r3**2
            b=-2*k[0]*k[1]*sin(theta[1])*cos(gamma)-2*k[1]**2*sin(theta[1])*cos(theta[1])
            c=k[1]**2*sin(theta[1])**2-r3**2
            delta=b**2-4*a*c
            if delta>err:
                y1=arctan((-b+sqrt(delta))/(2*a))
                y2=arctan((-b-sqrt(delta))/(2*a))
                # Python能这么写吗
                x1=y1+pi*(y1<0)
                x2=y2+pi*(y2<0)
                if x1-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x1)
                    if temp>0:
                        ang[0]=x1
                        R[0]=temp
                        tag[0]=1
                if x2-theta[1]<err:
                    temp=k[0]*sin(phi[0]+x2)
                    if temp>0:
                        ang[1]=x2
                        R[1]=temp
                        tag[1]=1
                if y1>=0:
                    x1=y1+pi
                else:
                    x1=2*pi+y1
                if y2>=0:
                    x2=y2+pi
                else:
                    x2=2*pi+y2
                if pi+theta[1]<x1+err and x1<2*pi+err:
                    temp=k[0]*sin(phi[0]-x1)
                    if temp>0:
                        ang[2]=x1
                        R[2]=temp
                        tag[2]=1
                if pi+theta[1]<x2+err and x2<2*pi+err:
                    temp=k[0]*sin(phi[0]-x2)
                    if temp>0:
                        ang[3]=x2
                        R[3]=temp
                        tag[3]=1
            elif abs(delta)<err:
                y=arctan(-b/(2*a))
                x=y+pi*(y<0)
                if theta[1]<x+err and x<pi+err:
                    temp=k[0]*sin(phi[0]+x)
                    if temp>0:
                        ang[0]=x
                        R[0]=temp
                        tag[0]=1
                if y>=0:
                    x=y+pi
                else:
                    x=2*pi+y
                if theta[1]+pi<x+err and x<2*pi+err:
                    temp=k[0]*sin(phi[0]-x)
                    if temp>0:
                        ang[2]=x
                        R[2]=temp 
                        tag[2]=1
        break
    rho=R[tag==1]
    ang=ang[tag==1]+phi0
    # print(array([rho@cos(ang),rho@sin(ang)]),A)
    p=array([rho@cos(ang),rho@sin(ang)])+A
    return p

r=array([[98,112,105,98,112,105,98,112]]).T
err=1e-2
theta=pi/180*array([[40.10,80.21,119.75,159.86,199.96,240.07,280.17,320.28]]).T
P=hstack([r*cos(theta),r*sin(theta)]).T
Q=R*vstack([cos(2*pi/9*arange(1,9).T),sin(2*pi/9*arange(1,9).T)])
x=R*vstack([cos(2*pi/9*arange(101).T),sin(2*pi/9*arange(101).T)])
temp=sum(sqrt((P-Q)*(P-Q)),axis=1)
m=max(temp)
counter=0
plt.plot(x[0,:],x[1,:])
plt.plot([0,100],[0,0],'r*')
plt.plot(Q[0,:],Q[1,:],'ob')
plt.plot(P[0,:],P[1,:],'r*')
plt.title('初始状态，最大误差距离为%.2f'%m)
plt.show()

while True:
    i=where(temp==min(temp))
    i=i[0]
    P1=P
    for j in range(8):
        if j!=i:
            temp1=angle[i,j,:]
            temp1=temp1[0]
            d=position(array([0,0]),array([R,0]),P[:,i].flatten(),temp1[0],temp1[1],temp1[2])
            n=d.shape[0]
            if n>1:
                temp2=sum((d-Q[:,j])*(d-Q[:,j]))
                ind=argsort(temp2)
                d=d[ind]
            if len(d)!=0:
                P1[:,j]=d
            tag=sqrt(sum((P1-Q)*(P1-Q),axis=1))<temp
            if sum(tag):
                for k in tag:
                    if k==True:
                        P[k,:]=P1[k,:]
                # P=(P1-P)*tag+P
                counter+=1
            temp=sqrt(sum((P-Q)*(P-Q),axis=1))
            m=max(temp)
            print(counter,m)
    if counter==5:
        P=Q
        m=err
        break
plt.plot(x[0,:],x[1,:])
plt.plot([0,100],[0,0],'r*')
plt.plot(Q[0,:],Q[1,:],'ob')
plt.plot(P[0,:],P[1,:],'r*')
plt.title('最终状态，最大误差距离为%.2f'%m)
plt.show()